# Recovery

Now to use the backups we made earlier let's simulate a recovery scenario.

## Regular

Our use case is: 

The`pg_control` file which contains crucial information about last REDO point (discussed in detail in WAL lesson) is removed.

The db can't start without it!

Let's save it with a recover.

```BASH
# Stop and remove file
pg_ctlcluster 16 demo stop
su postgres -c "rm /var/lib/postgresql/16/demo/global/pg_control"
# Try to start again
pg_ctlcluster 16 demo start
tail -n 20 /var/log/postgresql/postgresql-16-demo.log
# Damm...
```

```BASH
# Let's recover
# First, remove all db files
su postgres -c "find /var/lib/postgresql/16/demo -mindepth 1 -delete"
ls /var/lib/postgresql/16/demo/ # Nothing
# Now let's restore
su postgres -c "pgbackrest --stanza=demo restore"
ls /var/lib/postgresql/16/demo/ # It's all here!
pg_ctlcluster 16 demo start # All good just do recovery startup based on pg_wal
tail -n 20 /var/log/postgresql/postgresql-16-demo.log
```

## Delta Recovery

Configuration:
`delta=y` Config File
`--delta` Run Time

It's possible and strongly advised to avoid deleting all the files for DB restore phase. With this option, files are checked via hash function check equality to backup and equal files are skipped.

## Selected DBs

Recover only selected DBs instead of all

Use Cases:
- Recover a specific DB in a different machine
- Performance - incremental recovery

## Particular point in time

Let's demonstrate a common PITR scenario -> redo a human error like drop important table

```BASH
# Log MOD (DLL + Modifications) statements to be able to determine when the DROP happened
su postgres
sed -i "s/#*log_statement.*/log_statement = 'mod'/" $DB_CONF_FILE
pg_ctlcluster 16 demo reload
```

```BASH
# Create table
pg_ctlcluster -D 16 demo reload
psql << EOM
    SET log_statement
    BEGIN; 
    DROP TABLE IF EXISTS important_table;
    CREATE TABLE important_table (message TEXT); 
    INSERT INTO important_table VALUES ('Important Data'); 
    COMMIT; 
    SELECT * FROM important_table;
EOM
```

```BASH
# Drop table
psql -c "DROP TABLE important_table;"
psql -c "SELECT * FROM important_table;" # error!
```

```BASH
# Recovery
## Find time of query
DROP_QUERY_TS=$(grep "DROP TABLE important_table;" /var/log/postgresql/postgresql-16-demo.log | tail -1 | cut -d" " -f1,2)
echo "Drop query ts is: $DROP_QUERY_TS"
RECOVERY_TS=$(date -d "$(date -d "$DROP_QUERY_TS") - 2 seconds" +"%Y-%m-%d %H:%M:%S")
echo "Recovery ts, right before drop happened is: $RECOVERY_TS"

## Stop Server
pg_ctlcluster 16 demo stop

## Restore until timestamp
pgbackrest --stanza=demo --delta \
    --type=time "--target=$RECOVERY_TS" \
    --target-action=promote \
    --log-level-console=info restore
# Type - time / lsn
# Target Action

## Start server
pg_ctlcluster 16 demo start
psql -c "SELECT * FROM important_table;" # We are saved!

## Checkout server log
cat /var/log/postgresql/postgresql-16-demo.log
```